# Imports and Data preperation

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install git+https://github.com/albumentations-team/albumentations.git
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

import albumentations as ag
import matplotlib.pyplot as plt
import skimage.data as skd
import torch
import torch.nn as nn
import torch.nn.functional as F
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import segmentation_models_pytorch as smp
import tqdm
import cv2 
import os

In [ ]:
import os

# create datasets
train_path = "../input/airbus-ship-detection/train_v2"
test_path = "../input/airbus-ship-detection/test_v2"
train_imagelist = os.listdir(train_path)
test_imagelist = os.listdir(test_path)


masks = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations_v2.csv')


img_masks = masks[masks['EncodedPixels'].notnull()]['ImageId'].tolist() # get none empty
img_masks_noships = masks[masks['EncodedPixels'].isnull()]['ImageId'].tolist() # get none empty
# masks = masks.iloc[img_masks] # set as training


# p1 = 8172
p1 = 7092
masks['ImageId'] = train_path + "/" + masks['ImageId'].astype(str)

hasships_paths = masks[masks['EncodedPixels'].notnull()]['ImageId'].tolist()        #[train_path + "/" + name  for name in tqdm.notebook.tqdm(train_imagelist) if name in img_masks] 
noships_paths =  masks[masks['EncodedPixels'].isnull()]['ImageId'].tolist() #[train_path + "/" + name  for name in tqdm.notebook.tqdm(train_imagelist) if name in img_masks_noships]
        
all_paths_train = hasships_paths #+ noships_paths


print(len(all_paths_train), " samples")

validx = all_paths_train[:p1]
testx = all_paths_train[p1:2*p1]
trainx = all_paths_train[2*p1:] 

num_hweight = len(hasships_paths) - 2*p1
num_nhweight = len(trainx) - num_hweight

hweight = 1.0/num_hweight
nhweight = 0 #0.0/num_nhweight

# prepare submission
subx = [test_path + "/" + name for name in test_imagelist]

print(f"training: {len(trainx)} validation {len(validx)} test {len(testx)}")

masks.head()

In [ ]:
print(f"number of images with ships: {len(hasships_paths)}")
print(f"number of images without ships: {len(noships_paths)}")

# Usefull Functions

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [ ]:
def to_tensor(x, **kwargs):
    # return x.transpose(2, 0, 1).astype('float32')
    return x.transpose(2, 1, 0).astype('float32')
    # return np.rollaxis(x, 2, 0).astype('float32') # other option
  
def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        ag.augmentations.transforms.Lambda(image=preprocessing_fn),
        ag.augmentations.transforms.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return ag.Compose(_transform)

class Dataset_ships(Dataset):
  def __init__(self, x, y, transforms = None, preprocessing = None):

    self.x = x
    self.y = y
    self.transforms = transforms
    self.preprocessing = preprocessing
    

  def __getitem__ (self, idx):
    
    
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")          # a CUDA device object
    
    x = cv2.imread(self.x[idx])
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    
    # Take the individual ship masks and create a single mask array for all ships
    ImageId = self.x[idx]#.split("/")[-1]
    img_masks = self.y.loc[self.y['ImageId'] == ImageId, 'EncodedPixels'].tolist()
 
    y = np.zeros((768, 768))
    

    if isinstance(img_masks[0], str):
      for mask in img_masks:
          y += rle_decode(mask)

#     y = ag.Resize(224, 224, p=1)(image=y)['image']

    if self.transforms != None:
      transform = self.transforms(image=x, mask=y)
      x = transform['image']
      y = transform['mask']

    y = np.expand_dims(y, 2).astype(np.byte)

    if self.preprocessing != None:
      sample = self.preprocessing(image=x, mask=y)
      x = sample['image']
      y = sample['mask']


    y = torch.tensor(y, device=device)
    x = torch.tensor(x, device=device)
        
    return x, y

  def __len__(self):
    # return len(self.x)
    return len(self.x)

In [ ]:
from segmentation_models_pytorch.utils.meter import AverageValueMeter
import sys

def run_epoch(epoch, dataloader, log_frequncy=100, threshold=None):

        epoch.on_epoch_start()
        
        logs = {}
        loss_meter = AverageValueMeter()
        metrics_meters = {metric.__name__: AverageValueMeter() for metric in epoch.metrics}
        per_batch_metrics = {metric_fn.__name__:[] for metric_fn in epoch.metrics} # create item to track metrics per batch
        per_batch_metrics.update({'loss': []})

        n = 0

        with tqdm.notebook.tqdm(dataloader, desc=epoch.stage_name, file=sys.stdout, disable=not (epoch.verbose)) as iterator:
            print(f"len iterator {len(iterator)}")
            for x, y in iterator:
                x, y = x.to(epoch.device), y.to(epoch.device)
                loss, y_pred = epoch.batch_update(x, y)
                
                if threshold is not None:
                    y_pred = (y_pred > threshold) * y_pred
                    y_pred[y_pred!=0] = 1
                
                # update loss logs
                loss_value = loss.cpu().detach().numpy()
                loss_meter.add(loss_value)
                loss_logs = {epoch.loss.__name__: loss_meter.mean}
                logs.update(loss_logs)

                # update metrics logs
                for metric_fn in epoch.metrics:
                    metric_value = metric_fn(y_pred, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)

                    if n % log_frequncy == 0:
                      per_batch_metrics[metric_fn.__name__].append(metric_value)

                if n % log_frequncy == 0:
                  per_batch_metrics['loss'].append(loss_value)

                metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
                logs.update(metrics_logs)

                if epoch.verbose:
                    s = epoch._format_logs(logs)
                    iterator.set_postfix_str(s)

                n += 1

        return logs, per_batch_metrics



# todo add arguments
def train_model(m, critation, optimizer, train_loader, valid_loader, test_dataloader, epoches=2, log_frequncy=100, scheduler=None):
  device = None
  if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object

  if device != None:
    m.cuda()

  print(f"device: {device}, {next(m.parameters()).is_cuda }")

  metrics = [
      smp.utils.metrics.IoU(threshold=0.5),
      smp.utils.metrics.Accuracy(),
      smp.utils.metrics.Recall(),
      smp.utils.metrics.Precision(),
      smp.utils.metrics.Fscore()
  ]

  train_epoch = smp.utils.train.TrainEpoch(
      m, 
      loss=critation, 
      metrics=metrics, 
      optimizer=optimizer,
      device=device,
      verbose=True,
  )

  valid_epoch = smp.utils.train.ValidEpoch(
      m, 
      loss=critation, 
      metrics=metrics, 
      device=device,
      verbose=True,
  )

  # train model for some epochs

  max_score = 0
  train_logs = []
  valid_logs = []
  train_metrics_logs = []

  for i in range(0, epoches):
    
    print('\nEpoch: {}'.format(i))
    # train_logs.append(train_epoch.run(train_loader)) # the lines beneth should replace this
    # valid_logs.append(valid_epoch.run(valid_loader))
    
    # todo test to see if works
    train_log, train_metrics = run_epoch(train_epoch, train_loader, log_frequncy)
    valid_log, valid_metrics = run_epoch(valid_epoch, valid_loader, log_frequncy)
    train_metrics_logs.append(train_metrics)
    train_logs.append(train_log)
    valid_logs.append(valid_log)


  # evaluate model on test set
  test_epoch = smp.utils.train.ValidEpoch(
      model=m,
      loss=critation,
      metrics=metrics,
      device=device,
  )
    
  test_logs = test_epoch.run(test_dataloader)
    
  return [train_logs, valid_logs, test_logs], train_metrics_logs


In [ ]:
def show_examples(model, loader, num, threshold = None, ensemble=False):

  itr = iter(loader)
  x, t = next(itr)

  xs = []
  ys = []
  ts = []
  if ensemble:
    for m in model:
        m.eval()
  else:
    model.eval()
  with torch.no_grad():
    
    for j in range(num):
      x, t = next(itr)
    

      y = model(x)
      y = np.array(y.cpu()).squeeze()

      x = np.array(x.cpu()).squeeze()
      x = np.rollaxis(x,0,3)
      x = x + np.abs(x.min())
      x = (x/x.max())

      
      t = np.array(t.cpu()).squeeze()
      
      if threshold != None:
        y = (y > threshold) * y
        y[y!=0] = 1
    
      xs.append(x)
      ys.append(y)
      ts.append(t)

    fig, axarr = plt.subplots(num, 4, figsize=(20, 45))
    fig.tight_layout()
    for i in range(0,num):
      
      axarr[i][0].axis('off')
      axarr[i][1].axis('off')
      axarr[i][2].axis('off')
      axarr[i][3].axis('off')

      axarr[i][0].set_title("image")
      axarr[i][1].set_title("detection")
      axarr[i][2].set_title("ground truth")
      axarr[i][3].set_title("overlay")

      axarr[i][0].imshow(xs[i])
      axarr[i][1].imshow(ys[i])
      axarr[i][2].imshow(ts[i])
      axarr[i][3].imshow(xs[i])
      axarr[i][3].imshow(ys[i], alpha=0.4)



In [ ]:
class Dataset_ships_submission(Dataset):
  def __init__(self, x, y, transforms = None, preprocessing = None):

    self.x = x
    self.y = y
    self.transforms = transforms
    self.preprocessing = preprocessing

  def __getitem__ (self, idx):
    
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")          # a CUDA device object
    
    x = cv2.imread(self.x[idx])
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    
    # Take the individual ship masks and create a single mask array for all ships
    ImageId = self.x[idx].split("/")[-1]

    if self.transforms != None:
      transform = self.transforms(image=x)
      x = transform['image']

    if self.preprocessing != None:
      sample = self.preprocessing(image=x)
      x = sample['image']

    x = torch.tensor(x, device=device)
        
    return x, ImageId

  def __len__(self):
    # return len(self.x)
    return len(self.x)

In [ ]:
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


In [ ]:
checkpoint = torch.load('../input/airbus-ships-weights-danielk/linkcheckpoint0.31.pth.tar', map_location=torch.device('cuda'))
print(checkpoint.keys())
model = checkpoint['model']
optimizer = checkpoint['optimizer']
critation = checkpoint['critation']


# Training Loop

In [ ]:
ENCODER = 'resnet101'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['ship']
ACTIVATION = None # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

model = smp.Linknet(
    encoder_name=ENCODER, 
    # encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

model = model.cuda()

agu = ag.Compose([ag.RandomBrightness(p=0.5),ag.RandomRotate90(p=0.5), ag.RandomCrop(256,256,p=1)], p=1)
agu_valid = ag.Compose([], p=1)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER)#, ENCODER_WEIGHTS)
preprocess = get_preprocessing(preprocessing_fn)

dataset_train = Dataset_ships(trainx, masks, agu, preprocess)
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True) # sampler=weighted_sampler)

dataset_valid = Dataset_ships(validx, masks, None, preprocess)
valid_loader = DataLoader(dataset_valid, batch_size=32, shuffle=True)

dataset_test = Dataset_ships(testx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=1, shuffle=True)

critation = smp.utils.losses.BCEWithLogitsLoss()
optimizer = torch.optim.Adam([
      dict(params=model.parameters(), lr=0.0015),
  ])



model = model.cuda()
average_per_epoch_logs, train_metrics_logs = train_model(model, critation, optimizer, train_loader, valid_loader, test_dataloader, 2, 10)

***Saving model state***

In [ ]:
state = {
             'model': model,
             'optimizer': optimizer,
             'critation':critation}
filename = 'model.pth.tar'
torch.save(state, filename)

torch.save(model, "model.pth")

# Validation Loop on a trained model

In [ ]:

ENCODER = 'resnet101'

model = torch.load("../input/airbus-ships-weights-danielk/checkpoint0.77_unet.pth.tar")['model']

critation = smp.utils.losses.BCEWithLogitsLoss()

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER) #, ENCODER_WEIGHTS)
preprocess = get_preprocessing(preprocessing_fn)

dataset_valid = Dataset_ships(validx, masks, None, preprocess)
valid_loader = DataLoader(dataset_valid, batch_size=16, shuffle=True)

dataset_test = Dataset_ships(testx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=16, shuffle=True)

metrics = [
     smp.utils.metrics.IoU(threshold=0.5),
     smp.utils.metrics.Accuracy(),
     smp.utils.metrics.Recall(),
     smp.utils.metrics.Precision(),
     smp.utils.metrics.Fscore()
]

valid_epoch = smp.utils.train.ValidEpoch(
      model,
      loss=critation,
      metrics=metrics,
      device='cuda',
      verbose=True,
  )

model.to('cuda')
model.eval
valid_log, valid_metrics = run_epoch(valid_epoch, test_dataloader, 20, threshold=None)

# View Training graphs

In [ ]:
train_patch_graphs = [j for kk in train_metrics_logs for j in kk['loss'] ]
train_patch_graphs2 = [j for kk in train_metrics_logs for j in kk['accuracy'] ]
train_patch_graphs3 = [j for kk in train_metrics_logs for j in kk['iou_score'] ]
train_patch_graphs4 = [j for kk in train_metrics_logs for j in kk['fscore'] ]
train_patch_graphs5 = [j for kk in train_metrics_logs for j in kk['recall'] ]

In [ ]:
plt.plot(train_patch_graphs)

In [ ]:
plt.plot(train_patch_graphs4)

In [ ]:
plt.plot(train_patch_graphs3)

In [ ]:
plt.plot(train_patch_graphs2)

# Ensemble Models

In [ ]:
checkpoint = torch.load('../input/airbus-ships-weights-danielk/linkcheckpoint0.48.pth.tar', map_location=torch.device('cuda'))
print(checkpoint.keys())
model1 = checkpoint['model']
model1 = model1.cuda()
model1 = model1.eval()

checkpoint = torch.load('../input/airbus-ships-weights-danielk/checkpoint0.77_unet.pth.tar', map_location=torch.device('cuda'))
print(checkpoint.keys())
model2 = checkpoint['model']
model2 = model2.cuda()
model2 = model2.eval()

checkpoint = torch.load('../input/airbus-ships-weights-danielk/checkpoint0.67_new.pth.tar', map_location=torch.device('cuda'))
print(checkpoint.keys())
model3 = checkpoint['model']
model3 = model3.cuda()
model3 = model3.eval()

In [ ]:
class ensemble(torch.nn.Module):
    
    def __init__(self, models, threshold=None, late_threshold=None):
        super().__init__()
        self.models = models
        self.threshold = threshold
        self.late_threshold = late_threshold

    def forward(self, x):
        y = self.models[0](x)
        
        if self.threshold is not None:
            y = (y > self.threshold[0]) * y
            y[y!=0] = 1
        
        for i in range(1,len(self.models)):
            
            cy = self.models[i](x)
            
            if self.threshold is not None:
              cy = (cy > self.threshold[i]) * cy
              cy[cy!=0] = 1
            
            y = y + cy
            
        if self.threshold is not None:
            y[y > len(self.models)/2] = 1
            y[y!=0]=1
            
        elif self.late_threshold is not None:
             y = y/len(self.models)
             y = (y > self.late_threshold) * y
             y[y!=0] = 1
                
        else:
            y = y/len(self.models)
            
        
        return y

In [ ]:
#showing examples
ENCODER = 'resnet101'

# enmodel = ensemble([model, model2, model3], None, 0)  # uncomment for average ensemble
enmodel = ensemble([model, model2, model3], [0,0,0])  # uncomment to enable majority vote ensemble 

# enmodel = enmodel.cuda()

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER)#, ENCODER_WEIGHTS)
preprocess = get_preprocessing(preprocessing_fn)

agu = ag.Compose([ag.RandomBrightnessContrast(p=1) ,ag.Rotate(p=0.5)], p=1)

dataset_test = Dataset_ships(testx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=1, shuffle=True)

# model.cuda()
show_examples(enmodel, test_dataloader, 15, threshold=None, ensemble=False)



# Validation on Ensemble Model

In [ ]:

ENCODER = 'resnet101'

model = torch.load("../input/airbus-ships-weights-danielk/checkpoint0.77_unet.pth.tar")['model']

critation = smp.utils.losses.BCEWithLogitsLoss()

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER) #, ENCODER_WEIGHTS)
preprocess = get_preprocessing(preprocessing_fn)

dataset_valid = Dataset_ships(validx, masks, None, preprocess)
valid_loader = DataLoader(dataset_valid, batch_size=16, shuffle=True)

dataset_test = Dataset_ships(testx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=16, shuffle=True)

metrics = [
     smp.utils.metrics.IoU(threshold=0.5),
     smp.utils.metrics.Accuracy(),
     smp.utils.metrics.Recall(),
     smp.utils.metrics.Precision(),
     smp.utils.metrics.Fscore()
]

valid_epoch = smp.utils.train.ValidEpoch(
      enmodel,
      loss=critation,
      metrics=metrics,
      device='cuda',
      verbose=True,
  )

model.to('cuda')
model.eval
valid_log, valid_metrics = run_epoch(valid_epoch, test_dataloader, 20, threshold=None)

# Submission 

In [ ]:
subx = [test_path + "/" + name for name in test_imagelist]
ENCODER = 'resnet101'

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER)#, ENCODER_WEIGHTS)
preprocess = get_preprocessing(preprocessing_fn)

dataset_test = Dataset_ships_submission(subx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=1, shuffle=True)

# show_examples(model, test_dataloader, 3)
itr = iter(test_dataloader)

out_pred_rows = []

enmodel.eval()
for i in tqdm.notebook.tqdm(range(len(itr))):
  x, imgid = next(itr)
  imgid = imgid[0]

  enmodel.to('cuda')
  y = enmodel(x)
  y = y.cpu().detach().numpy().squeeze()

  c_rle = rle_encode(y)
  out_pred_rows += [{'ImageId': imgid, 'EncodedPixels': c_rle}]

submission_df = pd.DataFrame(out_pred_rows)[['ImageId', 'EncodedPixels']]
submission_df.to_csv('submission.csv', index=False)


In [ ]:
dataset_test = Dataset_ships(testx, masks, None, preprocess)
test_dataloader = DataLoader(dataset_test, batch_size=1, shuffle=True)
show_examples(enmodel,test_dataloader,10, None)